In [ ]:
import opt_note.scsp as scsp
from ortools.sat.python import cp_model

In [ ]:
import marimo as mo
import nbformat

# `AUTOMATON_CPSAT` モデルの改良

`Alphabet` アルゴリズムで求めた巨大な共通超配列の部分列の中から最短な共通超配列を見つける方針.

ただし `Alphabet` アルゴリズムの部分列には本当の最短共通超配列は存在しない場合がある.
例えば `ba`, `cb` の最短共通超配列は `cba` だが `Alphabet` アルゴリズムで構築した解は `abcabc` であり, この中から最短な共通超配列を探すと `bcab` のようになってしまう.
これは `Alphabet` アルゴリズムにおいてアルファベットを並べる順序が固定されているからであり, アルファベットの並びをブロックごとに可変にしたら改善するんじゃないかと考えた.

ただしアルファベットの並びを可変にすれば最適解を実行可能領域に含むようになるかは証明していない.
従って改良モデルにおける dual bound は (あくまで最初に用意した大きな配列の部分配列の中での dual bound なため) 実際には最適解より大きくなってしまう可能性がある.

In [ ]:
class Model:
    def __init__(self, instance: list[str], perm: bool = False):
        chars = "".join(sorted(list(set("".join(instance)))))
        max_len = len(chars) * max(len(s) for s in instance)

        cpmodel = cp_model.CpModel()

        if perm:
            cvars = [
                cpmodel.new_int_var(lb=0, ub=len(chars) - 1, name="") for _ in range(max_len)
            ]
        else:
            cvars = [
                cpmodel.new_constant(idx % len(chars)) for idx in range(max_len)
            ]

        valids = [cpmodel.new_bool_var("") for _ in cvars]
        transition_expressions = [
            cpmodel.new_int_var(lb=0, ub=len(chars), name="") for _ in range(max_len)
        ]

        # 初期解としてアルファベットアルゴリズムを設定
        if perm:
            for idx, cvar in enumerate(cvars):
                cpmodel.add_hint(cvar, idx % len(chars))
        for valid in valids:
            cpmodel.add_hint(valid, 1)
        for idx, texp in enumerate(transition_expressions):
            cpmodel.add_hint(texp, idx % len(chars) + 1)

        if perm:
            for t in range(max(len(s) for s in instance)):
                cpmodel.add_all_different(cvars[t * len(chars) : (t + 1) * len(chars)])

        for cvar, valid, texp in zip(cvars, valids, transition_expressions):
            cpmodel.add(texp == 0).only_enforce_if(~valid)
            cpmodel.add(texp == cvar + 1).only_enforce_if(valid)

        for s in instance:
            transition_triples = (
                [
                    (idx, jdx + 1, (idx + 1 if c == next_char else idx))
                    for idx, next_char in enumerate(s)
                    for jdx, c in enumerate(chars)
                ]
                + [(idx, 0, idx) for idx, _ in enumerate(s)]
                + [(len(s), 0, len(s))]
                + [(len(s), jdx + 1, len(s)) for jdx, _ in enumerate(chars)]
            )
            cpmodel.add_automaton(
                transition_expressions=transition_expressions,
                starting_state=0,
                final_states=[len(s)],
                transition_triples=transition_triples,
            )

        cpmodel.minimize(sum(valids))

        self.instance = instance
        self.chars = chars
        self.cpmodel = cpmodel
        self.cpsolver = cp_model.CpSolver()
        self.cvars = cvars
        self.valids = valids
        self.status: cp_model.cp_model_pb2.CpSolverStatus | None = None

    def solve(self, time_limit: int | None = 60, log: bool = False) -> "Model":
        self.cpsolver.parameters.log_search_progress = log
        if time_limit is not None:
            self.cpsolver.parameters.max_time_in_seconds = time_limit
        self.status = self.cpsolver.solve(self.cpmodel)

        return self

    def to_solution(self) -> str | None:
        if self.status not in {
            cp_model.cp_model_pb2.OPTIMAL,
            cp_model.cp_model_pb2.FEASIBLE,
        }:
            return None

        solution = ""
        for cvar, valid in zip(self.cvars, self.valids):
            if self.cpsolver.boolean_value(valid):
                cidx = self.cpsolver.value(cvar)
                solution += self.chars[cidx]

        return solution

In [ ]:
def bench1(instance: list[str]) -> None:
    model = scsp.model.automaton_cpsat.Model(instance).solve()
    solution = model.to_solution()
    scsp.util.show(instance)
    if solution is not None:
        scsp.util.show(instance, solution)
        print(f"solution is feasible: {scsp.util.is_feasible(instance, solution)}")
    else:
        print("--- Solution not found ---\n")

    print(f"solution status: {model.cpsolver.status_name()}")
    print(f"bset bound: {model.cpsolver.best_objective_bound}")

In [ ]:
def bench2(instance: list[str], perm: bool) -> None:
    model = Model(instance, perm).solve()
    solution = model.to_solution()
    scsp.util.show(instance)
    if solution is not None:
        scsp.util.show(instance, solution)
        print(f"solution is feasible: {scsp.util.is_feasible(instance, solution)}")
    else:
        print("--- Solution not found ---\n")

    print(f"solution status: {model.cpsolver.status_name()}")
    print(f"bset bound: {model.cpsolver.best_objective_bound}")

最初に示した例で計算してみよう.
`ba`, `cb` に対する最短共通超配列を `AUTOMATON_CPSAT` モデルで求めると最適解 (の 1 つ) は `cba` だとわかる.

In [ ]:
bench1(["ba", "cb"])

--- Condition (with 3 chars) ---
str1: ba
str2: cb

--- Solution (of length 3) ---
 Sol: cba
str1: -ba
str2: cb-

solution is feasible: True
solution status: OPTIMAL
bset bound: 3.0


一方順番を固定した改良法だとこの最適解は出ない.

In [ ]:
bench2(["ba", "cb"], perm=False)

--- Condition (with 3 chars) ---
str1: ba
str2: cb

--- Solution (of length 4) ---
 Sol: bcab
str1: b-a-
str2: -c-b

solution is feasible: True
solution status: OPTIMAL
bset bound: 4.0


アルファベットの順番を可変にすると元の最適解が出るようになる.

In [ ]:
bench2(["ba", "cb"], perm=True)

--- Condition (with 3 chars) ---
str1: ba
str2: cb

--- Solution (of length 3) ---
 Sol: cba
str1: -ba
str2: cb-

solution is feasible: True
solution status: OPTIMAL
bset bound: 3.0


## ベンチマーク

In [ ]:
instance01 = scsp.example.load("uniform_q26n004k015-025.txt")

In [ ]:
bench1(instance01)

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 67) ---
 Sol: iutkglcojiqfolenbkxxycvosuhamqopvixnozhtsqgxbzprplpsvddbxicsvrvnngf
str1: --tkg----------n-k-------uh-m--p--xn--ht-qgx-z------v---xi-s-------
str2: i------ojiqfol-nb-xx-cv-su---q-pvi------s----------s---bx---------f
str3: -u---lc--i-----n----yc-os-----o-v---oz--------p-plp----------------
str4: i---g---------e-------v----a---------z----g-b--r-----ddb--csvrvnngf

solution is feasible: True
solution status: FEASIBLE
bset bound: 11.0


In [ ]:
bench2(instance01, perm=False)

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 64) ---
 Sol: iogjeiqtvafkozglnbkruxdhlmpxcdinvybchostuoqvgoxzprvxipslpsbnnxgf
str1: -------t---k--g-n-k-u--h-mpx---n----h--t--q-g-xz--vxi-s---------
str2: io-j-iq---f-o--lnb---x-----xc---v-----s-u-q-----p-v-i-s--sb--x-f
str3: --------------------u---l---c-in-y-c-os--o-v-o-zp----p-lp-------
str4: i-g-e---va---zg--b-r--d------d----bc--s----v-----rv--------nn-gf

solution is feasible: True
solution status: FEASIBLE
bset bound: 50.0


In [ ]:
bench2(instance01, perm=True)

--- Condition (with 25 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf

--- Solution (of length 75) ---
 Sol: iotugjklceginqvyacfkosuzghlmnopvxbnorxzdhptxcdpqvbcglsuvxzqrvxinpvinsgsfbxf
str1: --t---k---g-n------k--u--h-m--p-x-n-----h-t----q---g----xz--vxi-----s------
str2: io---j-----i-q----f-o-----l-n----b---x-----xc---v----su---q-----pvi-s-s-bxf
str3: ---u---lc--in--y-c--os-------o-v---o--z--p----p-----l-----------p----------
str4: i---g----e----v-a------zg--------b--r--d-----d---bc--s-v---rv--n---n-g-f---

solution is feasible: True
solution status: FEASIBLE
bset bound: 3.0


In [ ]:
instance02 = scsp.example.load("uniform_q26n008k015-025.txt")

In [ ]:
bench1(instance02)

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 137) ---
 Sol: tkgnkuhmpxnhtqgiojiqfolnbxxcvsuqplcvisngycoesovazgbrddbcplsvorzvnnxucpmqvgtdfeuivdcvdpfzsmsbroenqvbczfjtvxerzbrvigphwxqkrdrlctodtmprpxwed
str1: tkgnkuhmpxnhtqg----------x----------------------z----------v------x------------i--------s------------------------------------------------
str2: ---------------iojiqfolnbxxcvsuqp--vis------s-----b---------------x---------f------------------------------------------------------------
str3: -----u----------------l----c--------i-n-yco-sov-------------o-z------p---------------p-------------------------------------l------p------
str4: ---------------i-----------------------g---e--vazgbrddbc--sv-r-vnn-------g--f------------------------------------------

In [ ]:
bench2(instance02, perm=False)

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 112) ---
 Sol: iugloprtxbdejnvbcwxadiknqyzcfjotvdglnpbfklrxzdrxcdlsuvbcempsuvbhmqrtopvxgtdfnuhistozbmpqrvinbcgpxzvwxdisblpxefho
str1: -------t--------------k-----------g-n---k-----------u----------hm----p-x----n-h--t-----q------g-xzv-x-is--------
str2: i---o-------j--------i--q---f-o----ln-b----x---xc----v-----su----q---pv--------is----------------------sb--x-f--
str3: -u-l------------c----i-n-y-c--o--------------------s----------------o-v-----------oz--p--------p---------lp-----
str4: i-g--------e--v----a------z-------g---b---r--d---d----bc---s-v----r---v-----n--------------n--g--------------f--
str5: -----p-------------------y-----------p---lr-z--x----u--c--p-----mq----v-gtdf-u-i---------v---c------

In [ ]:
bench2(instance02, perm=True)

--- Condition (with 26 chars) ---
str1: tkgnkuhmpxnhtqgxzvxis
str2: iojiqfolnbxxcvsuqpvissbxf
str3: ulcinycosovozpplp
str4: igevazgbrddbcsvrvnngf
str5: pyplrzxucpmqvgtdfuivcdsbo
str6: pbdevdcvdpfzsmsbroqvbbh
str7: enbczfjtvxerzbrvigple
str8: rxwxqkrdrlctodtmprpxwd

--- Solution (of length 289) ---
 Sol: iogqrsetvxayzbcdefghijklmnopqrstuvwxyzbcdefghijklmnopqvwxyzbcdefghijklmnopqrstuvwxyzbcdefghijklmnopqrstavwxyubrdefghijlcnopqustyvwxmzabcdefgqijkxtuvgeorstucvxyzabcdefghijklmnopqrstuvwxyzabcdefghijklmnsqbcdefghijklmnopqrvwxzabcdefghijklmoqpstuvwxyzarstuvwxyzabcdefghijklmnopqrstuvwxyzabwxhf
str1: -------t--------------k--------------------g------n-----------------k---------u-----------h----m--p-------x-------------n----------------------------------------------h-----------t---------------------q-----g-------------xz-------------------v-x--------------------i---------s-------------
str2: io-------------------j-----------------------i-------q---------f--------o---------------------l-n------

In [ ]:
instance03 = scsp.example.load("uniform_q26n016k015-025.txt")

In [ ]:
bench1(instance03)

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 241) ---
  Sol: tkgnkuhmpxnhiojiqfolnbxxulcinycosoigevoazgbrddbcsvpyplrzxucpmqvgtbdevdfucvdpfzsmsnbczfjtvxerzbrvigxwxqkrdrlctodtmpkkqafigqjwokkskrblxxpabivbvzkozrifsavnqaxudgqvqcewbfgijowwyrsxqjnfpadiusiqbwezshkvhcomiuvddhtxxqjzqbctbakxusfcfzpeecvwantfmgqzu
str01: tkgnkuhmpxnh----------------------------------------------------t------------------------------------q------------------g-----------x--------z--------v---x------------i------s--------------------------------------------------

In [ ]:
bench2(instance03, perm=False)

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution (of length 164) ---
  Sol: ioxhjktuwgikpqrxyesvafipwxzghnoqubdejklnrvxzbdgqrwxdfpuabcfhpsvzdilmpquxacefjotvzgknwbehkmstycimoqsxdefgruxzbeijkqtmoprvwzabchkozhlnptxdfimnqsvbgpqszbhlopceuvwxdfmy
str01: ------t----k---------------g-n-------k----------------u----h-------mp--x-----------n---h---t-----q-----g--xz-----------v--------------x--i---s----------------------
str02: io--j-----i--q-------f--------o-------ln----b-----x--------------------x-c-----v----------s--------------u-------q---p-v----------

In [ ]:
bench2(instance03, perm=True)

--- Condition (with 26 chars) ---
str01: tkgnkuhmpxnhtqgxzvxis
str02: iojiqfolnbxxcvsuqpvissbxf
str03: ulcinycosovozpplp
str04: igevazgbrddbcsvrvnngf
str05: pyplrzxucpmqvgtdfuivcdsbo
str06: pbdevdcvdpfzsmsbroqvbbh
str07: enbczfjtvxerzbrvigple
str08: rxwxqkrdrlctodtmprpxwd
str09: kkqafigqjwokkskrblg
str10: lxxpabivbvzkozzvd
str11: krifsavncdqwhzc
str12: qaxudgqvqcewbfgijowwy
str13: rsxqjnfpadiusiqbezhkohmg
str14: iwshvhcomiuvddm
str15: htxxqjzqbctbakn
str16: xusfcfzpeecvwantfmgqzu

--- Solution not found ---

solution status: UNKNOWN
bset bound: 0.0


アルファベットの順番を可変にすると解が悪化した.

## ログを見よう

In [ ]:
_model = Model(instance01)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0xcdd8bbc2799f944b)
#Variables: 1'275 (#bools: 625 in objective) (1'275 primary variables)
  - 625 Booleans in [0,1]
  - 625 in [0,25]
  - 25 constants in {0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24} 
#kAutomaton: 4
#kLinear1: 625 (#enforced: 625)
#kLinear2: 625 (#enforced: 625)

Starting presolve at 0.01s
The solution hint is complete and is feasible. Its objective value is 625.


  1.87e-04s  0.00e+00d  [DetectDominanceRelations] 
  9.77e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  2.71e-06s  0.00e+00d  [ExtractEncodingFromLinear] 


  4.11e-03s  0.00e+00d  [DetectDuplicateColumns] 


  1.81e-02s  0.00e+00d  [DetectDuplicateConstraints] #duplicates=78'735 


[Symmetry] Graph for symmetry has 213'916 nodes and 561'604 arcs.


...

#Bound  85.48s best:64    next:[51,63]    bool_core (num_cores=51 [size:3 mw:1 d:13] a=364 d=13 fixed=49/59755 clauses=53'557)


#Bound  95.19s best:64    next:[52,63]    bool_core (num_cores=52 [size:17 mw:1 amo:2 lit:10 d:9] a=353 d=13 fixed=49/59805 clauses=52'325)


#Bound  96.69s best:64    next:[53,63]    bool_core (num_cores=53 [size:111 mw:1 amo:9 lit:67 d:10] a=257 d=13 fixed=49/59919 clauses=53'206)


#Bound 112.98s best:64    next:[54,63]    bool_core (num_cores=54 [size:1 mw:1] a=257 d=13 fixed=49/60073 clauses=54'991)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [   1.95m,    1.95m]    1.95m   0.00ns    1.95m         1 [   1.22m,    1.22m]    1.22m   0.00ns    1.22m
           'default_lp':         1 [   1.95m,    1.95m]    1.95m   0.00ns    1.95m         1 [   1.25m,    1.25m]    1.25m   0.00ns    1.25m
     'feasibility_pump':       508 [ 62.39us, 349.33ms] 885.13us  15.48ms 449.65ms       507 [  4.34us,   4.34us]   4.34us   0.00ns   2.20ms
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
            'fs_random':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

In [ ]:
_model = Model(instance02)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0xc7a082dbe57f8289)
#Variables: 1'326 (#bools: 650 in objective) (1'326 primary variables)
  - 650 Booleans in [0,1]
  - 650 in [0,26]
  - 26 constants in {0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25} 
#kAutomaton: 8
#kLinear1: 650 (#enforced: 650)
#kLinear2: 650 (#enforced: 650)

Starting presolve at 0.00s
The solution hint is complete and is feasible. Its objective value is 650.


  2.48e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.40e-01s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  3.14e-06s  0.00e+00d  [ExtractEncodingFromLinear] 


  2.13e-02s  0.00e+00d  [DetectDuplicateColumns] 


  4.57e-02s  0.00e+00d  [DetectDuplicateConstraints] #duplicates=177'015 


[Symmetry] Graph for symmetry has 474'700 nodes and 1'254'896 arcs.


...

#Bound  64.13s best:109   next:[58,108]   bool_core (num_cores=58 [size:5 mw:1 d:5] a=119 d=7 fixed=206/131765 clauses=85'323)


#Bound  68.06s best:109   next:[59,108]   bool_core (num_cores=59 [size:6 mw:1 d:8] a=114 d=8 fixed=206/131779 clauses=86'469)


#Bound  73.19s best:109   next:[60,108]   bool_core (num_cores=60 [size:8 mw:1 d:8] a=107 d=8 fixed=206/131822 clauses=87'617)


#Bound  95.74s best:109   next:[61,108]   bool_core (num_cores=61 [size:5 mw:1 d:9] a=103 d=9 fixed=206/131854 clauses=93'114)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [   1.89m,    1.89m]    1.89m   0.00ns    1.89m         1 [  51.40s,   51.40s]   51.40s   0.00ns   51.40s
           'default_lp':         1 [   1.89m,    1.89m]    1.89m   0.00ns    1.89m         1 [   1.06m,    1.06m]    1.06m   0.00ns    1.06m
     'feasibility_pump':       477 [ 94.46us, 410.61ms]   1.17ms  18.77ms 557.68ms       456 [ 54.00ns,  54.00ns]  54.00ns   0.00ns  24.62us
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
            'fs_random':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

In [ ]:
_model = Model(instance03)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x165220b78a585bd9)
#Variables: 1'326 (#bools: 650 in objective) (1'326 primary variables)
  - 650 Booleans in [0,1]
  - 650 in [0,26]
  - 26 constants in {0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25} 
#kAutomaton: 16
#kLinear1: 650 (#enforced: 650)
#kLinear2: 650 (#enforced: 650)

Starting presolve at 0.00s
The solution hint is complete and is feasible. Its objective value is 650.


  3.41e-04s  0.00e+00d  [DetectDominanceRelations] 
  2.60e-01s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  3.19e-06s  0.00e+00d  [ExtractEncodingFromLinear] 


  3.67e-02s  0.00e+00d  [DetectDuplicateColumns] 


  9.15e-02s  0.00e+00d  [DetectDuplicateConstraints] #duplicates=352'449 


[Symmetry] Graph for symmetry has 942'310 nodes and 2'495'980 arcs.


...

#Bound 110.82s best:157   next:[57,156]   bool_core (num_cores=57 [size:4 mw:1 d:5] a=103 d=6 fixed=1086/260290 clauses=163'747)


#Bound 112.23s best:157   next:[58,156]   bool_core (num_cores=58 [size:4 mw:1 d:6] a=100 d=6 fixed=1086/260301 clauses=163'913)


#Bound 113.27s best:157   next:[59,156]   bool_core (num_cores=59 [size:2 mw:1 d:5] a=99 d=6 fixed=1086/260317 clauses=164'054)


#Bound 117.02s best:157   next:[60,156]   bool_core (num_cores=60 [size:5 mw:1 d:6] a=95 d=6 fixed=1086/260331 clauses=164'628)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [   1.78m,    1.78m]    1.78m   0.00ns    1.78m         1 [  40.09s,   40.09s]   40.09s   0.00ns   40.09s
           'default_lp':         1 [   1.78m,    1.78m]    1.78m   0.00ns    1.78m         1 [  48.53s,   48.53s]   48.53s   0.00ns   48.53s
     'feasibility_pump':       419 [ 80.69us, 709.06ms]   2.03ms  34.59ms 849.09ms       409 [ 18.00ns,  18.00ns]  18.00ns   0.00ns   7.36us
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
            'fs_random':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

In [ ]:
instance04 = scsp.example.load("uniform_q05n010k010-010.txt")

In [ ]:
_model = Model(instance04)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x36a98f53e50a01f8)
#Variables: 105 (#bools: 50 in objective) (105 primary variables)
  - 50 Booleans in [0,1]
  - 50 in [0,5]
  - 5 constants in {0,1,2,3,4} 
#kAutomaton: 10
#kLinear1: 50 (#enforced: 50)
#kLinear2: 50 (#enforced: 50)

Starting presolve at 0.00s
The solution hint is complete and is feasible. Its objective value is 50.


  3.60e-05s  0.00e+00d  [DetectDominanceRelations] 
  4.64e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  1.19e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  2.59e-04s  0.00e+00d  [DetectDuplicateColumns] 
  7.95e-04s  0.00e+00d  [DetectDuplicateConstraints] #duplicates=4'356 
[Symmetry] Graph for symmetry has 18'830 nodes and 48'407 arcs.


[Symmetry] Symmetry computation done. time: 0.00351307 dtime: 0.0071441
[Symmetry] #generators: 28, average support size: 2
[Symmetry] The model contains 10 duplicate constraints !
[Symmetry] 8 orbits on 36 variables with sizes: 10,8,6,4,2,2,2,2
[Symmetry] Found orbitope of size 1 x 6


[SAT presolve] num removable Booleans: 314 / 4917
[SAT presolve] num trivial clauses: 0
[SAT presolve] [0s] clauses:13732 literals:34028 vars:4879 one_side_vars:0 simple_definition:0 singleton_clauses:0
[SAT presolve] [0.000715767s] clauses:13722 literals:33998 vars:4879 one_side_vars:0 simple_definition:10 singleton_clauses:0
[SAT presolve] [0.00109404s] clauses:13624 literals:33998 vars:4830 one_side_vars:0 simple_definition:10 singleton_clauses:0


  2.15e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  8.96e-02s  3.40e-01d  [Probe] #probed=9'836 #fixed_bools=9 #equiv=55 #new_binary_clauses=9'891 


...

#Bound   1.00s best:30    next:[23,29]    bool_core (num_cores=23 [size:3 mw:1 d:6] a=13 d=7 fixed=0/4560 clauses=3'851) [skipped_logs=2]


#Bound   1.07s best:30    next:[28,29]    max_lp [skipped_logs=2]


#15      2.18s best:29    next:[28,28]    pseudo_costs


#Done    2.49s bool_core (num_cores=28 [size:5 mw:1 exo] a=0 d=9 fixed=3/4622 clauses=16'063)
#Done    2.49s core



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [   2.06s,    2.06s]    2.06s   0.00ns    2.06s         1 [   3.15s,    3.15s]    3.15s   0.00ns    3.15s
           'default_lp':         1 [   2.06s,    2.06s]    2.06s   0.00ns    2.06s         1 [   2.51s,    2.51s]    2.51s   0.00ns    2.51s
     'feasibility_pump':        15 [ 32.10us,  26.52ms]   1.81ms   6.60ms  27.16ms        14 [558.00ns, 558.00ns] 558.00ns   0.00ns   7.81us
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
            'fs_random':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

In [ ]:
instance05 = scsp.example.load("uniform_q05n050k010-010.txt")

In [ ]:
_model = Model(instance05)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0xd54b493ab9d279de)
#Variables: 105 (#bools: 50 in objective) (105 primary variables)
  - 50 Booleans in [0,1]
  - 50 in [0,5]
  - 5 constants in {0,1,2,3,4} 
#kAutomaton: 50
#kLinear1: 50 (#enforced: 50)
#kLinear2: 50 (#enforced: 50)

Starting presolve at 0.00s
The solution hint is complete and is feasible. Its objective value is 50.


  6.97e-05s  0.00e+00d  [DetectDominanceRelations] 
  9.60e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  1.08e-06s  0.00e+00d  [ExtractEncodingFromLinear] 


  3.96e-03s  0.00e+00d  [DetectDuplicateColumns] 
  6.39e-03s  0.00e+00d  [DetectDuplicateConstraints] #duplicates=21'474 


[Symmetry] Graph for symmetry has 91'413 nodes and 236'551 arcs.


[Symmetry] Symmetry computation done. time: 0.0276285 dtime: 0.0389261
[Symmetry] #generators: 244, average support size: 2
[Symmetry] The model contains 50 duplicate constraints !
[Symmetry] 9 orbits on 253 variables with sizes: 50,40,38,33,28,25,21,11,7


...

#Model  42.67s var:21352/22758 constraints:18431/19104


#Model  43.82s var:21351/22758 constraints:18431/19104


#Model  48.64s var:20828/22758 constraints:18249/19104


#6      51.40s best:34    next:[]         core
#Done   51.40s core



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [  49.41s,   49.41s]   49.41s   0.00ns   49.41s         1 [  32.89s,   32.89s]   32.89s   0.00ns   32.89s
           'default_lp':         1 [  49.42s,   49.42s]   49.42s   0.00ns   49.42s         1 [  30.62s,   30.62s]   30.62s   0.00ns   30.62s
     'feasibility_pump':       209 [ 16.05us, 416.71ms]   2.06ms  28.75ms 431.40ms       208 [  1.05us,   1.05us]   1.05us   0.00ns 218.40us
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
            'fs_random':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

In [ ]:
instance06 = scsp.example.load("nucleotide_n010k010.txt")

In [ ]:
_model = Model(instance06)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0xf915dff1f4e91e99)
#Variables: 84 (#bools: 40 in objective) (84 primary variables)
  - 40 Booleans in [0,1]
  - 40 in [0,4]
  - 4 constants in {0,1,2,3} 
#kAutomaton: 10
#kLinear1: 40 (#enforced: 40)
#kLinear2: 40 (#enforced: 40)

Starting presolve at 0.00s
The solution hint is complete and is feasible. Its objective value is 40.
  2.60e-05s  0.00e+00d  [DetectDominanceRelations] 
  1.55e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  1.57e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  1.69e-04s  0.00e+00d  [DetectDuplicateColumns] 
  5.47e-04s  0.00e+00d  [DetectDuplicateConstraints] #duplicates=2'697 
[Symmetry] Graph for symmetry has 13'133 nodes and 33'522 arcs.


[Symmetry] Symmetry computation done. time: 0.00243195 dtime: 0.00501712
[Symmetry] #generators: 39, average support size: 2
[Symmetry] The model contains 10 duplicate constraints !
[Symmetry] 7 orbits on 46 variables with sizes: 10,7,7,6,6,6,4
[Symmetry] Found orbitope of size 1 x 6
[SAT presolve] num removable Booleans: 239 / 3403
[SAT presolve] num trivial clauses: 0
[SAT presolve] [0s] clauses:9575 literals:23609 vars:3356 one_side_vars:0 simple_definition:0 singleton_clauses:0
[SAT presolve] [0.000502884s] clauses:9565 literals:23579 vars:3356 one_side_vars:0 simple_definition:10 singleton_clauses:0
[SAT presolve] [0.00080136s] clauses:9465 literals:23579 vars:3306 one_side_vars:0 simple_definition:10 singleton_clauses:0


  9.12e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  5.26e-02s  1.93e-01d  [Probe] #probed=6'826 #equiv=39 #new_binary_clauses=7'647 


  1.69e-02s  5.94e-02d  [MaxClique] Merged 6'762(13'524 literals) into 2'382(8'992 literals) at_most_ones. 
  7.45e-04s  0.00e+00d  [DetectDominanceRelations] 
  3.68e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  4.14e-04s  0.00e+00d  [ProcessAtMostOneAndLinear] 
  1.84e-04s  0.00e+00d  [DetectDuplicateConstraints] #duplicates=71 
  1.36e-04s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  3.76e-05s  0.00e+00d  [DetectDominatedLinearConstraints] 
  3.90e-05s  0.00e+00d  [DetectDifferentVariables] 
  1.95e-03s  9.36e-05d  [ProcessSetPPC] #relevant_constraints=5'368 #num_inclusions=2'392 
  5.36e-05s  0.00e+00d  [FindAlmostIdenticalLinearConstraints] 
  6.74e-04s  6.15e-04d  [FindBigAtMostOneAndLinearOverlap] 
  1.05e-04s  6.47e-05d  [FindBigVerticalLinearOverlap] 
  4.12e-05s  0.00e+00d  [FindBigHorizontalLinearOverlap] 
  1.51e-04s  4.25e-06d  [MergeClauses] #num_collisions=150 #num_merges=150 #num_saved_literals=350 
  5.17e-04s  

  5.17e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.66e-03s  0.00e+00d  [PresolveToFixPoint] #num_loops=1 #num_dual_strengthening=1 
  1.96e-04s  0.00e+00d  [DetectDuplicateColumns] 
  9.47e-05s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 7'042 nodes and 13'319 arcs.
[Symmetry] Symmetry computation done. time: 0.00128296 dtime: 0.0026479
[Symmetry] #generators: 12, average support size: 8
[Symmetry] 20 orbits on 68 variables with sizes: 5,5,5,5,4,4,4,4,3,3,...
[Symmetry] Found orbitope of size 4 x 5
[SAT presolve] num removable Booleans: 0 / 3248
[SAT presolve] num trivial clauses: 0
[SAT presolve] [0s] clauses:250 literals:850 vars:231 one_side_vars:0 simple_definition:212 singleton_clauses:0
[SAT presolve] [2.3454e-05s] clauses:250 literals:850 vars:231 one_side_vars:0 simple_definition:212 singleton_clauses:0
[SAT presolve] [8.3438e-05s] clauses:250 literals:850 vars:231 one_side_vars:0 simple_definition:212 singleton_clauses:0


...

#Model   0.85s var:2948/2956 constraints:2592/2595


#Bound   1.25s best:24    next:[23,23]    bool_core (num_cores=22 [size:2 mw:1 exo] a=0 d=9 fixed=10/3101 clauses=9'691)


#Model   1.28s var:2860/2956 constraints:2553/2595


#Done    1.42s core



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [   1.18s,    1.18s]    1.18s   0.00ns    1.18s         1 [   1.94s,    1.94s]    1.94s   0.00ns    1.94s
           'default_lp':         1 [   1.18s,    1.18s]    1.18s   0.00ns    1.18s         1 [   1.43s,    1.43s]    1.43s   0.00ns    1.43s
     'feasibility_pump':        12 [ 38.25us,  17.45ms]   1.50ms   4.81ms  17.98ms        11 [648.00ns, 648.00ns] 648.00ns   0.00ns   7.13us
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
            'fs_random':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

In [ ]:
instance07 = scsp.example.load("nucleotide_n050k050.txt")

In [ ]:
_model = Model(instance07)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x8cd991355f0c11fb)
#Variables: 505 (#bools: 250 in objective) (505 primary variables)
  - 250 Booleans in [0,1]
  - 250 in [0,5]
  - 5 constants in {0,1,2,3,4} 
#kAutomaton: 50
#kLinear1: 250 (#enforced: 250)
#kLinear2: 250 (#enforced: 250)

Starting presolve at 0.00s
The solution hint is complete and is feasible. Its objective value is 250.


  3.11e-04s  0.00e+00d  [DetectDominanceRelations] 
  1.85e-01s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  1.84e-06s  0.00e+00d  [ExtractEncodingFromLinear] 


  7.22e-02s  0.00e+00d  [DetectDuplicateColumns] 


  1.40e-01s  0.00e+00d  [DetectDuplicateConstraints] #duplicates=459'333 


[Symmetry] Graph for symmetry has 1'819'534 nodes and 4'884'866 arcs.
[Symmetry] Graph too large. Skipping. You can use symmetry_level:3 or more to force it.


...

#Model  94.03s var:495154/495252 constraints:381159/381217


#Bound  97.90s best:138   next:[14,137]   bool_core (num_cores=14 [size:15 mw:1 d:4] a=54 d=5 fixed=49/495616 clauses=332'688)


#Bound 112.67s best:138   next:[15,137]   bool_core (num_cores=15 [size:2 mw:1 d:5] a=53 d=5 fixed=98/495630 clauses=332'664)


#Bound 114.20s best:138   next:[16,137]   bool_core (num_cores=15 [cover] a=53 d=5 fixed=98/495643 clauses=332'746)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [   1.61m,    1.61m]    1.61m   0.00ns    1.61m         1 [  31.91s,   31.91s]   31.91s   0.00ns   31.91s
           'default_lp':         1 [   1.61m,    1.61m]    1.61m   0.00ns    1.61m         1 [  38.89s,   38.89s]   38.89s   0.00ns   38.89s
     'feasibility_pump':       362 [ 19.57us,    1.64s]   5.22ms  86.20ms    1.89s       361 [ 18.00ns,  18.00ns]  18.00ns   0.00ns   6.50us
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
            'fs_random':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

In [ ]:
instance08 = scsp.example.load("protein_n010k010.txt")

In [ ]:
_model = Model(instance08)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0xe864effb1f6f5f8)
#Variables: 399 (#bools: 190 in objective) (399 primary variables)
  - 190 Booleans in [0,1]
  - 190 in [0,19]
  - 19 constants in {0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18} 
#kAutomaton: 10
#kLinear1: 190 (#enforced: 190)
#kLinear2: 190 (#enforced: 190)

Starting presolve at 0.00s
The solution hint is complete and is feasible. Its objective value is 190.


  8.22e-05s  0.00e+00d  [DetectDominanceRelations] 
  1.92e-02s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  1.47e-06s  0.00e+00d  [ExtractEncodingFromLinear] 


  1.24e-03s  0.00e+00d  [DetectDuplicateColumns] 
  5.45e-03s  0.00e+00d  [DetectDuplicateConstraints] #duplicates=24'747 


[Symmetry] Graph for symmetry has 74'600 nodes and 190'840 arcs.


[Symmetry] Symmetry computation done. time: 0.0151266 dtime: 0.0304396
[Symmetry] #generators: 170, average support size: 4.56471
[Symmetry] The model contains 10 duplicate constraints !
[Symmetry] 83 orbits on 390 variables with sizes: 100,10,10,9,8,7,7,7,5,5,...


...

#32     42.37s best:47    next:[43,46]    quick_restart


#Bound  57.13s best:47    next:[44,46]    bool_core (num_cores=44 [size:13 mw:1 d:15] a=53 d=15 fixed=454/19498 clauses=44'258)


#Model  83.65s var:18288/19079 constraints:14527/14919


#Bound  98.50s best:47    next:[45,46]    bool_core (num_cores=45 [size:18 mw:1 amo:2 lit:6 d:16] a=37 d=16 fixed=454/19559 clauses=45'726)



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [   1.98m,    1.98m]    1.98m   0.00ns    1.98m         1 [   1.42m,    1.42m]    1.42m   0.00ns    1.42m
           'default_lp':         1 [   1.98m,    1.98m]    1.98m   0.00ns    1.98m         1 [   1.37m,    1.37m]    1.37m   0.00ns    1.37m
     'feasibility_pump':       521 [ 23.08us, 182.29ms] 444.28us   7.97ms 231.47ms       519 [  1.19us,   1.19us]   1.19us   0.00ns 616.57us
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
            'fs_random':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

In [ ]:
instance09 = scsp.example.load("protein_n050k050.txt")

In [ ]:
_model = Model(instance09)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x4bc14be872f6a772)
#Variables: 2'020 (#bools: 1'000 in objective) (2'020 primary variables)
  - 1'000 Booleans in [0,1]
  - 1'000 in [0,20]
  - 20 constants in {0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19} 
#kAutomaton: 50
#kLinear1: 1'000 (#enforced: 1'000)
#kLinear2: 1'000 (#enforced: 1'000)

Starting presolve at 0.00s
The solution hint is complete and is feasible. Its objective value is 1000.


  1.16e-03s  0.00e+00d  [DetectDominanceRelations] 
  2.23e+00s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  3.98e-06s  0.00e+00d  [ExtractEncodingFromLinear] 


  5.14e-01s  0.00e+00d  [DetectDuplicateColumns] 


  1.11e+00s  0.00e+00d  [DetectDuplicateConstraints] #duplicates=3'166'290 


[Symmetry] Problem too large. Skipping. You can use symmetry_level:3 or more to force it.


...

#2     115.54s best:980   next:[0,979]    rnd_cst_lns (d=5.00e-01 s=9 t=0.10 p=0.00 stall=0 h=base) [hint]


#3     119.63s best:979   next:[0,978]    no_lp [hint]


#Model 121.88s var:2385900/2386139 constraints:1482089/1482214



Task timing                      n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                 'core':         1 [  11.22s,   11.22s]   11.22s   0.00ns   11.22s         1 [631.24ms, 631.24ms] 631.24ms   0.00ns 631.24ms
           'default_lp':         1 [  11.18s,   11.18s]   11.18s   0.00ns   11.18s         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
     'feasibility_pump':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                   'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
            'fs_random':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
      'fs_ra

CpSolverResponse summary:
status: FEASIBLE
objective: 979
best_bound: 0
integers: 0
booleans: 0
conflicts: 0
branches: 0
propagations: 0
integer_propagations: 0
restarts: 0
lp_iterations: 0
walltime: 130.422
usertime: 130.422
deterministic_time: 36.2181
gap_integral: 4.86433
solution_fingerprint: 0xc2224aa698994a72



性能自体は悪くないかも.
例えば `protein_n050k050.txt` で 497 は `WMM_HEXALY` を 1 だけ上回っている.

一方で巨大なインスタンスでは presolve に時間がかかりすぎていて初期解をちょっと改善して終わりみたいになったりする.
DNA 配列の長さ 100 を 100 個用意して計算してみると...

In [ ]:
instance_large1 = scsp.example.load("nucleotide_n100k100.txt")

In [ ]:
_model = Model(instance_large1)
_model.solve(time_limit=120, log=True)
_model.to_solution()


Starting CP-SAT solver v9.14.6206
Parameters: max_time_in_seconds: 120 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0xa753fb3562db7aba)
#Variables: 1'206 (#bools: 600 in objective) (1'206 primary variables)
  - 600 Booleans in [0,1]
  - 600 in [0,6]
  - 6 constants in {0,1,2,3,4,5} 
#kAutomaton: 100
#kLinear1: 600 (#enforced: 600)
#kLinear2: 600 (#enforced: 600)

Starting presolve at 0.00s
The solution hint is complete and is feasible. Its objective value is 600.


  1.28e-03s  0.00e+00d  [DetectDominanceRelations] 
  1.91e+00s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  2.58e-06s  0.00e+00d  [ExtractEncodingFromLinear] 


  9.72e-01s  0.00e+00d  [DetectDuplicateColumns] 


  2.17e+00s  0.00e+00d  [DetectDuplicateConstraints] #duplicates=4'783'638 


[Symmetry] Problem too large. Skipping. You can use symmetry_level:3 or more to force it.


...

  1.40e+01s  1.95e+01d *[MaxClique] Merged 9'703'442(19'425'787 literals) into 9'698'495(19'420'440 literals) at_most_ones. 


272 affine relations still in the model.



Presolve summary:
  - 1553 affine relations were detected.
  - rule 'TODO dual: only one blocking constraint?' was applied 50 times.
  - rule 'affine: new relation' was applied 1'553 times.
  - rule 'at_most_one: transformed into max clique.' was applied 2 times.
  - rule 'automaton: expanded' was applied 100 times.
  - rule 'bool_and: x => x' was applied 100 times.
  - rule 'deductions: 1200 stored' was applied 1 time.
  - rule 'dual: fix variable' was applied 1 time.
  - rule 'duplicate: removed constraint' was applied 4'783'638 times.
  - rule 'exactly_one: removed literals' was applied 75 times.
  - rule 'exactly_one: singleton' was applied 75 times.
  - rule 'linear: always true' was applied 1'100 times.
  - rule 'linear: enforcement literal in expression' was applied 1'100 times.
  - rule 'linear: fixed or dup variables' was applied 1'100 times.
  - rule 'linear: remapped using affine relations' was applied 61'100 times.
  - rule 'new_bool: automaton expansion' was applied 4'822

Presolved optimization model '': (model_fingerprint: 0x3e61f363711afec3)
#Variables: 4'781'963 (#bools: 599 in objective) (4'722'347 primary variables)
  - 4'781'963 Booleans in [0,1]
#kAtMostOne: 5'898 (#literals: 35'246)
#kBoolAnd: 604'474 (#enforced: 604'474) (#literals: 10'297'071)
#kBoolOr: 2'833'902 (#literals: 13'043'421)
#kExactlyOne: 59'344 (#literals: 4'468'213)
#kLinear2: 272


Stopped after presolve.
PresolvedNumVariables: 4781963
PresolvedNumConstraints: 3503890
PresolvedNumTerms: 27843807
CpSolverResponse summary:
status: UNKNOWN
objective: 0
best_bound: 0
integers: 0
booleans: 0
conflicts: 0
branches: 0
propagations: 0
integer_propagations: 0
restarts: 0
lp_iterations: 0
walltime: 127.683
usertime: 127.683
deterministic_time: 42.3856
gap_integral: 0



presolve が終わらなかった...